## Survival Analysis "Company Names"

### Libraries

In [ ]:
import pandas as pd
import numpy as np

from lifelines import *
from lifelines.statistics import logrank_test, multivariate_logrank_test
from lifelines.utils import to_episodic_format, restricted_mean_survival_time
from lifelines.plotting import rmst_plot
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

### Data Import

In [ ]:
data = pd.read_excel(r"data.xlsx")
D, E = data['Duration'], data['Event']

### Kaplan-Meier-Curves

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(D, E)

In [ ]:
fig, ax =  plt.subplots()
kmf.plot_survival_function(color='C0',ax=ax, at_risk_counts=True)
ax.set(
    title='Kaplan-Meier survival curve',
    xlabel='Years',
    ylabel='Estimated Probability of Survival')

In [ ]:
kmf.median_survival_time_

In [ ]:
fig, ax = plt.subplots()
L = ['Without Semantic Endowment', 'With Semantic Endowment']
for r in data['Semantic_Endowment'].unique():
    ix = data[data['Semantic_Endowment'] == r]
    kmf.fit(ix['Duration'], ix['Event'], label = L[r])
    kmf.plot(ax=ax)

ax.set(
    title='Kaplan-Meier Survival Curves Semantic Endowment',
    xlabel='Operational / Years',
    ylabel='Estimated Probability of Survival'
)

# likewise: other variables

### Predictor Set

In [ ]:
formula = "Fluency + Semantic_Endowment + Symbolic_Endowment + Legal_Form + \
           Rubrication + Complexity + Legal_Form*Semantic_Endowment" # last term = interaction

### Multicollinearity Test

In [ ]:
def calc_vif(X):

    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

feature_vif = calc_vif(data[['Fluency', 'Rubrication', 'Semantic_Endowment', 'Complexity', 
                              'Symbolic_Endowment', 'Legal_Form']])
feature_vif.sort_values('VIF')

### Proportional Hazard Assumption Test

In [ ]:
cph = CoxPHFitter()
cph.fit(data, 'Duration', 'Event', formula=formula)
cph.print_summary(decimals=3)
cph.check_assumptions(data, show_plots=True, p_value_threshold=0.05)

In [ ]:
cph.plot()

### Logrank Test

In [ ]:
results = multivariate_logrank_test(D, data['Business_Model'], E)
results.print_summary(decimals=3, style=None)

### Transformation to Episodic Format 

In [ ]:
long = to_episodic_format(data, 'Duration', 'Event', time_gaps=1) # time_gaps = intervals

# long.to_excel('long.xlsx')

In [ ]:
long1 = long[['start', 'stop', 'id', 'Event', 'Legal_Form', 'Rubrication', 'Semantic_Endowment', 
              'Symbolic_Endowment', 'Complexity', 'Fluency']]
long1 = long1.astype('float')

In [ ]:
long1['t_Rubrication'] = long1['Rubrication'] * long1['stop'] # Rubrication violates PH-Assumption
long1.drop(['Rubrication'], axis=1, inplace=True)

In [ ]:
formula_episodic = "Fluency + Semantic_Endowment + Symbolic_Endowment + Legal_Form + \
                    t_Rubrication + Complexity + Legal_Form*Semantic_Endowment"

### Cox Regression

In [ ]:
from lifelines import CoxTimeVaryingFitter
ctv = CoxTimeVaryingFitter()

ctv.fit(long1,
        id_col='id',
        event_col='Event',
        start_col='start',
        stop_col='stop', formula = formula_episodic)

In [ ]:
ctv.print_summary(3, model="age * time interaction")

In [ ]:
ctv.plot()

### Factorizing

In [ ]:
long2 = long[['start', 'stop', 'id', 'Event', 'Legal_Form', 'Rubrication', 'Semantic_Endowment', 
             'Symbolic_Endowment', 'Complexity', 'Fluency', 'Business_Model', 'Value_Creation']]

In [ ]:
long2['t_Rubrication'] = long2['Rubrication'] * long2['stop']
long2.drop(['Rubrication'], axis=1, inplace=True)

In [ ]:
for r in long2['Business_Model'].unique():
    print(r)
    df = long2[long2['Business_Model'] == r]
    df = df.astype('float')

    ctv.fit(df, id_col='id', event_col='Event', start_col='start', stop_col='stop', formula=formula_episodic)

    ctv.print_summary(3)

In [ ]:
for r in long2['Value_Creation'].unique():
    print(r)
    df = long2[long2['Value_Creation'] == r]
    df = df.astype('float')

    ctv.fit(df, id_col='id', event_col='Event', start_col='start', stop_col='stop', formula=formula_episodic)

    ctv.print_summary(3)

### Median Survival Time (MST) & Restricted Median Survival Time (RMST)

In [ ]:
value = [1,0]
for val in value:
    df = data[data['Rubrication']==val]
    kmf.fit(df['Duration'], df['Event'])
    print('rubrication =', val, kmf.median_survival_time_)

In [ ]:
ix = data['Semantic_Endowment'] == 1

time_limit = 10

kmf_exp = KaplanMeierFitter().fit(D[ix], E[ix], label='explanatory names')
rmst_exp = restricted_mean_survival_time(kmf_exp, t=time_limit)

kmf_con = KaplanMeierFitter().fit(D[~ix], E[~ix], label='non-explanatory names')
rmst_con = restricted_mean_survival_time(kmf_con, t=time_limit)

In [ ]:
ax = plt.subplot()
rmst_plot(kmf_exp, model2=kmf_con, t=time_limit, ax=ax)

plt.legend(loc='upper right')
plt.show()